# Prompt Engineering



In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import openai
from openai import OpenAI
import os
from getpass import getpass

In [3]:
openai.api_key = getpass("OPENAI_API_KEY: ")

OPENAI_API_KEY: ··········


#### helper function

We will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs:

In [4]:
client = OpenAI(
    api_key=openai.api_key
)

In [9]:
def get_completion(message,
                   model="gpt-3.5-turbo",
                   temperature=0):

    response = client.chat.completions.create(
        model = model,
        messages = [
          {
              "role": "user",
              "content": message,
          }],
        temperature = temperature
    )

    return response.choices[0].message.content

## Prompting guidelines




### Write clear and specific instructions



#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [15]:
text = f"""
Machine learning is a field of study that focuses on developing algorithms and models that allow \
computers to learn from and make predictions or decisions based on data. It involves training \
a model on a dataset, evaluating its performance, and using it to make predictions on new, unseen data.
"""

prompt = f"""
Summarize the given text into a single sentence.
```{text}```
"""

response = get_completion(prompt, temperature = 1.5)
print(response)

Machine learning is the study of developing algorithms for computers to learn from and make predictions or decisions based on data.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [16]:
prompt = f"""
Generate a list of three large cap companies in healthcare sector in BSE.
Provide them in JSON format with the following keys:
Organzation name, BSE ticker.
"""
response = get_completion(prompt)
print(response)

{
  "companies": [
    {
      "Organization name": "Sun Pharmaceutical Industries Ltd.",
      "BSE ticker": "SUNPHARMA"
    },
    {
      "Organization name": "Dr. Reddy's Laboratories Ltd.",
      "BSE ticker": "DRREDDY"
    },
    {
      "Organization name": "Cipla Ltd.",
      "BSE ticker": "CIPLA"
    }
  ]
}


#### Tactic 4: "Few-shot" prompting

In [17]:
prompt = f"""
Sample1: The Pt has a pain-scale is 6/10. Classify the patiet as Uncomplicated or Complicated ? Answer: Uncomplicated
Sample2: The Pt has a pain-scale is 7/10. Classify the patiet as Uncomplicated or Complicated ? Answer: Complicated
Sample3: The Pt pain-scale is at 3/10. Is he Uncomplicated or Complicated ? Answer:
"""
response = get_completion(prompt)
print(response)

Uncomplicated


#### Tactic 1: Specify the steps required to complete a task

In [18]:
text = f"""
Chandrayaan-3 was launched from Satish Dhawan Space Centre on 14 July 2023. The spacecraft entered \
lunar orbit on 5 August, and the lander touched down near the Lunar south pole on  23 August at 18:03 \
IST (12:33 UTC), making India the fourth country to successfully land on the Moon, and the first to do \
so near the lunar south pole. On 3 September the lander hopped and repositioned itself 30–40 cm (12–16 in) \
from its landing site making the second hop on lunar surface after Surveyor 6. After the completion of its \
mission objectives, it was hoped that the lander and rover would revive for extra tasks, on 22 September \
2023, but missed the wake-up call. On September 30, the second lunar night began, eliminating hopes of \
revival. On the other hand, Propulsion module returned to Earth's orbit on 22 November 2023 after exiting \
lunar  orbit via a series of maneuvers starting from 9 October 2023.
"""

# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text in 1 sentence.
2 - Translate the summary into Hindi.
3 - Output a JSON object that contains important event and their dates.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - Chandrayaan-3 was launched on 14 July 2023, successfully landed near the lunar south pole on 23 August, and the lander and rover failed to revive for extra tasks on 22 September 2023.
2 - चंद्रयान-3 को 14 जुलाई 2023 को लॉन्च किया गया था, 23 अगस्त को चंद्रमा के दक्षिणी ध्रुव के पास सफलतापूर्वक उतरा और 22 सितंबर 2023 को लैंडर और रोवर को अतिरिक्त कार्यों के लिए जागृत करने में असफल रहा।
3 - {
  "events": [
    {
      "event": "Launch",
      "date": "14 July 2023"
    },
    {
      "event": "Landing",
      "date": "23 August 2023"
    },
    {
      "event": "Failed Revival",
      "date": "22 September 2023"
    },
    {
      "event": "Propulsion Module Return",
      "date": "22 November 2023"
    }
  ]
}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [20]:
prompt = """
Evaluate the accuracy of the student's mathematical derivation.

Question:
A group of friends decides to embark on a road trip adventure. They plan to rent a car and share the expenses equally. The car rental company offers a daily rental rate of $50 and an additional charge of $0.25 per mile driven. Each friend contributes an equal amount towards the total expenses. If the group drives a total of 'm' miles during the trip, what is the cost per person as a function of 'm'?

Student's Solution:
Let 'm' represent the total miles driven.
Cost Calculation:
1. Daily rental cost: $50
2. Mileage cost: $0.25 * m
Total cost: $50 + ($0.25 + m)
Cost per person: ($50 + ($0.25 + m)) / number_of_friends

Evaluate the mathematical derivation provided by the student and determine if it accurately calculates the cost per person as a function of the total miles driven.
"""
# Total cost: $50 + ($0.25 '*' m) should be the correct value
response = get_completion(prompt)
print(response)

The student's mathematical derivation is not accurate. 

In the calculation of the total cost, the student incorrectly adds $0.25 and 'm' instead of multiplying them. The correct calculation should be $0.25 * m. 

Therefore, the correct cost per person as a function of 'm' should be:
Cost per person = ($50 + ($0.25 * m)) / number_of_friends


## Model Limitations: Hallucinations

Hallucination refers to the tendency of the models to produce text that appears to be correct but is actually false or not based on the input given.

- FutureTech is an original company. But XYZ Quantum is not its product.

In [21]:
prompt = prompt = f"""
Tell me about the latest smartphone model, the XYZ Quantum, developed by a futuristic technology company called FuturaTech.
"""
response = get_completion(prompt)
print(response)

The XYZ Quantum is the latest smartphone model developed by FuturaTech, a futuristic technology company known for its innovative and cutting-edge products. This smartphone is designed to push the boundaries of technology and provide users with a truly futuristic experience.

One of the standout features of the XYZ Quantum is its advanced quantum computing capabilities. It is powered by a state-of-the-art quantum processor, which enables lightning-fast processing speeds and unparalleled computational power. This allows users to perform complex tasks and run resource-intensive applications with ease.

The XYZ Quantum boasts a stunning and immersive display. It features a large, bezel-less OLED screen with vibrant colors and sharp resolution. The display supports HDR10+ technology, providing a dynamic and lifelike visual experience. Whether you are watching movies, playing games, or browsing the web, the XYZ Quantum offers an exceptional viewing experience.

In terms of photography, the X

### Generate a marketing product description from a product fact sheet

In [22]:
fact_sheet = """
Brand	Apple
Model Name	IPhone
Network Service Provider	Unlocked for All Carriers
Operating System	IOS
Cellular Technology	5G
About this item
15.40 cm (6.1-inch) Super Retina XDR display
Advanced camera system for better photos in any light
Cinematic mode now in 4K Dolby Vision up to 30 fps
Action mode for smooth, steady, handheld videos
Vital safety technology — Crash Detection calls for help when you can't
All-day battery life and up to 20 hours of video playback
Industry-leading durability features with Ceramic Shield and water resistance
A15 Bionic chip with 5-core GPU for lightning-fast performance. Superfast 5G cellular
iOS 16 offers even more ways to personalise, communicate and share
"""

### Limit the number of words/sentences/characters.

In [23]:
prompt = f"""
Create a marketing summary for the below tech specifications

Use at most 50 words only.

Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt)
print(response)


Introducing the Apple iPhone, unlocked for all carriers. Experience the future with 5G technology and a 6.1-inch Super Retina XDR display. Capture stunning photos and videos with the advanced camera system and enjoy all-day battery life. With industry-leading durability and lightning-fast performance, the iPhone is the ultimate device for personalization and communication.


In [ ]:
val = response.split(" ") # get the word count by splitting white spaces.
print(val)

['Introducing', 'the', 'Apple', 'iPhone,', 'a', 'powerful', 'device', 'with', 'a', '6.1-inch', 'Super', 'Retina', 'XDR', 'display,', 'advanced', 'camera', 'system,', 'and', 'cinematic', 'mode', 'in', '4K', 'Dolby', 'Vision.', 'With', 'all-day', 'battery', 'life,', 'industry-leading', 'durability,', 'and', 'the', 'lightning-fast', 'A15', 'Bionic', 'chip,', "it's", 'the', 'perfect', 'companion', 'for', 'personalization,', 'communication,', 'and', 'sharing.', 'Experience', 'the', 'future', 'with', '5G', 'cellular', 'technology.']


In [ ]:
len(val)

52

### Generate content for target audience
- Ask it to focus on the aspects that are relevant to the intended audience.

In [24]:
prompt = f"""
Create a marketing summary for the below tech specifications
The description is intended for smartphone retailers,
so should be technical in nature and focus on the
camera system of the phone.

Use at most 50 words.

Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt)
print(response)

Introducing the Apple iPhone, featuring a 6.1-inch Super Retina XDR display and an advanced camera system for stunning photos in any lighting. Capture cinematic 4K Dolby Vision videos with ease, and enjoy smooth, steady action shots. With all-day battery life and industry-leading durability, this phone is powered by the lightning-fast A15 Bionic chip and supports superfast 5G cellular technology. Personalize, communicate, and share like never before with iOS 16.


## Text Summarization

In [25]:
prod_review = """

This laptop is amazing! It performs flawlessly and exceeds expectations.
The sleek design and lightweight make it perfect for work on the go.
The customer service from the seller was excellent. They promptly answered all
my queries before making the purchase, and the laptop arrived well-packaged and in perfect condition.
I also received the laptop earlier than expected, which was a pleasant surprise.
The battery life is impressive, and the display is vibrant.
Highly recommended!
"""

In [26]:
# Limiting the text

prompt = f"""
Summarize the below text in atmost 30 words.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


This review praises the laptop for its flawless performance, sleek design, lightweight, excellent customer service, prompt delivery, impressive battery life, vibrant display, and recommends it highly.


In [ ]:
# Focus on an aspect

prompt = f"""
Summarize the below text, in at most 20 words,
by focusing on any aspects \
that mention shipping and delivery of the product.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


The laptop arrived well-packaged and in perfect condition, and the customer received it earlier than expected.


In [ ]:
# Focus on an aspect

prompt = f"""
Generate a short summary of a product review text below \
to give feedback on the \
performance of the product.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


The reviewer is extremely satisfied with the laptop's performance, design, and customer service. They praise its flawless performance, sleek design, and lightweight, making it ideal for portable work. The seller's customer service is described as excellent, with prompt responses to queries and a well-packaged laptop delivered in perfect condition. The reviewer also appreciates the early arrival of the laptop, the impressive battery life, and the vibrant display. Overall, they highly recommend the product.


The review focuses on performance of product, customer service etc. and not just product.


In [ ]:
prompt = f"""
Generate a short summary of a product review text below \
to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


The customer is extremely satisfied with the laptop, praising its flawless performance, sleek design, and lightweight. They also appreciate the excellent customer service and prompt delivery. The battery life and display quality are impressive. Overall, the customer highly recommends the product.


Summaries include topics that are not related to the topic of focus.

## Sentiment Analysis

In [ ]:
review = """
I was in search of comfortable and stylish bata shoes for everyday wear, and these shoes fit the bill perfectly.
The design is trendy and versatile, making them suitable for various occasions. I received the shoes
promptly after placing the order. However, there was a small defect on one shoe. I contacted customer support,
and they were incredibly helpful and understanding. They immediately shipped a replacement pair, which arrived
within a few days. The shoes are incredibly comfortable, and I love how they fit. The company's dedication to
customer satisfaction is commendable. I highly recommend these shoes!
"""

In [ ]:
prompt = f"""
What is the sentiment of the below review ?

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is positive.


In [ ]:
prompt = f"""
What is the sentiment of the below review ?

Answer in one word, positive or negative.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

positive


## Emotion Tags

In [ ]:
prompt = f"""
List all the emotions that are expressed by the writer in
the below review text in a comma separated output?

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

satisfaction, comfort, love, commendable, helpful, understanding, trendy, versatile, prompt


## NER

In [ ]:
prompt = f"""
Get the following items from the review text:
- Item purchased by reviewer
- Company that made the item


Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

- Item purchased by reviewer: Bata shoes
- Company that made the item: Bata


## Topic Modeling

In [ ]:
story = """
In a recent survey on online shopping preferences, 85% of participants revealed that they
frequently shop online, citing convenience as the main motivator. Among online platforms,
60% preferred e-commerce marketplaces, while 25% opted for brand websites.
Mobile commerce dominated, with 70% using smartphones for shopping. Product reviews heavily influenced purchases,
with 80% relying on them. Same-day or next-day delivery was preferred by 45%, and 35% were willing to pay extra for
faster shipping. Prompt customer support via live chat or social media was expected by 65%. Concerns about data security
were raised by 30%. These insights emphasize convenience, mobile commerce, customer reviews, fast delivery, and
effective support as crucial aspects of successful online shopping experiences.

"""

In [ ]:
prompt = f"""
Give me five topics that are mentioned in the above story.
Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

online shopping, convenience, e-commerce marketplaces, mobile commerce, product reviews


In [ ]:
response.split(sep=',')

['online shopping',
 ' convenience',
 ' e-commerce marketplaces',
 ' mobile commerce',
 ' product reviews']

In [ ]:
topic_list = ['online shopping','convenience','e-commerce marketplaces','mobile commerce','product reviews','space tech','cricket']

Added couple of off-topics -- 'space tech','cricket'

Checking whether the topics are correctly classified

In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

[1, 1, 1, 1, 1, 0, 0]


## Translation

In [ ]:
prompt = f"""
Translate the following text English text to Hindi: \
`` Order pav bhaji for snacks today. ``
"""
response = get_completion(prompt)
print(response)

आज नाश्ते के लिए पाव भाजी आर्डर करें।


In [ ]:
prompt = f"""
Translate the following to Hindi in both the formal and informal forms:
'Would you like to order a burger?'
"""
response = get_completion(prompt)
print(response)

Formal: क्या आपको बर्गर आर्डर करना है? (Kya aapko burger order karna hai?)
Informal: क्या तुम्हें बर्गर आर्डर करना है? (Kya tumhein burger order karna hai?)


Multiple languages to a single language to help Customer support Team.

In [ ]:
user_messages = [
  "La connexion Internet est instable.",                       # The Internet connection is unstable
  "J'ai perdu mes données importantes.",                        # I have lost my important data
  #"Necesito ayuda para configurar mi correo electrónico."      # I need help setting up my email
]

In [ ]:
for message in user_messages:
    print(f"Original message : {message}")

    prompt = f"""
    Translate the following text to English and tell me which language it is in first place : ```{message}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message : La connexion Internet est instable.
The given text is in French. It translates to English as: "The Internet connection is unstable." 

Original message : J'ai perdu mes données importantes.
The given text is in French. It translates to English as: "I have lost my important data." 



In [ ]:
# Tone changes

prompt = f"""
Translate the following from slang to a business letter:
'Yaar, Rahul here! Check out the specs of this software application, ye katarnak hai!'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I hope this letter finds you well. I am writing to bring your attention to the specifications of a software application that I believe is exceptional.

The software application in question is truly remarkable and possesses outstanding features. Its capabilities are unparalleled and it is sure to revolutionize the industry. I highly recommend taking a closer look at its specifications.

Thank you for your time and consideration. I look forward to discussing this matter further with you.

Sincerely,

Joe


## Token Calculation

```
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played in Texas at Globe Life Field in Arlington.",
        "role": "assistant"
      }
    }
  ],
  "created": 1677664795,
  "id": "chatcmpl-7QyqpwdfhqwajicIEznoc6Q47XAyW",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 57,
    "total_tokens": 74
  }
}
```

In [ ]:
def get_completion_and_token_count(message,
                                   model="gpt-3.5-turbo",
                                   temperature=0,
                                   max_tokens=500):

    response = client.chat.completions.create(
        model = model,
        messages = [
          {
              "role": "user",
              "content": message,
          }],
        temperature = temperature
    )

    content = response.choices[0].message.content

    token_dict = {
      'prompt_tokens':response.usage.prompt_tokens,
      'completion_tokens':response.usage.completion_tokens,
      'total_tokens':response.usage.total_tokens,
    }

    return content, token_dict

In [ ]:
prompt = f"""
Translate the following to Hindi in both the formal and informal forms:
'Would you like to order a burger?'
"""
response, tokens_info = get_completion_and_token_count(prompt)

In [ ]:
print(response)

Formal: क्या आपको बर्गर आर्डर करना है? (Kya aapko burger order karna hai?)
Informal: क्या तुम्हें बर्गर आर्डर करना है? (Kya tumhein burger order karna hai?)


In [ ]:
print(tokens_info)

{'prompt_tokens': 31, 'completion_tokens': 91, 'total_tokens': 122}


## Moderation

[OpenAI Moderation API](https://platform.openai.com/docs/guides/moderation)

In [ ]:
response = client.moderations.create(input="The street brawl erupted between rival gangs, leading to multiple injuries.")

output = response.results[0]

print(output)

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=7.0982673605612945e-06, harassment_threatening=8.588772288931068e-07, hate=1.3330077308637556e-06, hate_threatening=4.967619648965638e-09, self_harm=8.286984432004374e-09, self_harm_instructions=6.613423865617016e-14, self_harm_intent=9.541157794390465e-12, sexual=6.912439687312144e-08, sexual_minors=4.791338437115655e-09, violence=0.004309084732085466, violence_graphic=4.1566603613318875e-05, self-harm=8.286984432004374e-09, sexual/minors=4.791338437115655e-09, hate/threatening=4.967619648965638e-09, violence/graphic=

## Chain of Thought Prompting

In [ ]:
system_message = f"""
Answer the customer queries based on the product info given below.

1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99
"""

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0):

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    return response.choices[0].message.content

In [ ]:
user_message = f"""by how much is the BlueWave Gaming Laptop more expensive than the MegaPro Desktop"""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{user_message}"},
]

response = get_completion_from_messages(messages)
print(response)

The BlueWave Gaming Laptop is $1199.99, while the TechPro Desktop is $999.99. Therefore, the BlueWave Gaming Laptop is $200 more expensive than the TechPro Desktop.


In [ ]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}.

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count.

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.

All available products:

1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information.

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [ ]:
user_message = f"""
by how much is the BlueWave Gaming Laptop more expensive than the MegaPro Desktop"""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about the price difference between the BlueWave Gaming Laptop and the MegaPro Desktop.

Step 2:#### The BlueWave Gaming Laptop and the MegaPro Desktop are not in the list of available products. Therefore, we cannot provide specific information about their prices.

Step 3:#### The user is assuming that the MegaPro Desktop is a product available for purchase.

Step 4:#### Since the MegaPro Desktop is not listed as one of the available products, we cannot provide information about its price or make any comparisons.

Response to user:#### I apologize, but I don't have information about the MegaPro Desktop as it is not one of the available products. If you have any questions about the listed products, I'll be happy to assist you.
